In [1]:
import pandas as pd
from openpyxl import load_workbook
from zipfile import BadZipFile

file_path = "C:/Users/olcay/Desktop/TASKS/TrakRacer/USA Free Shipping/TRONERATE.xlsx"

try:
    # Try loading the workbook
    wb = load_workbook(filename=file_path)
    df = pd.read_excel(file_path, engine='openpyxl')
    print("File loaded successfully!")
except BadZipFile:
    print("Error: The file is not a valid Excel file or is corrupted.")
except FileNotFoundError:
    print("Error: The file was not found at the specified path.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


File loaded successfully!


In [2]:


# Define the correct packaging criteria based on FedEx One Rate categories and the rates
def get_one_rate_category(weight, length, width, height):
    if weight > 50:  # Check if the package exceeds the weight limit for One Rate
        return "Not Eligible", 0

    volume = length * width * height
    if volume <= 300 and max(length, width, height) <= 15.5:
        return "FedEx Envelope", 7.19  # Rate for Envelope
    elif volume <= 650 and max(length, width, height) <= 15.5:
        return "FedEx Pak", 9.32  # Rate for Pak
    elif volume <= 420 and max(length, width, height) <= 12.375:
        return "FedEx Small Box", 10.41  # Rate for Small Box
    elif volume <= 650 and max(length, width, height) <= 13.25:
        return "FedEx Medium Box", 12.97  # Rate for Medium Box
    elif volume <= 1100 and max(length, width, height) <= 17.5:
        return "FedEx Large Box", 17.64  # Rate for Large Box
    elif volume <= 2200 and max(length, width, height) <= 15.75:
        return "FedEx Extra Large Box", 23.59  # Rate for Extra Large Box
    else:
        return "Not Eligible", 0



In [3]:
# Apply the One Rate category and cost function to the DataFrame
df['One Rate Category'], df['One Rate Cost'] = zip(*df.apply(
    lambda row: get_one_rate_category(row[' GW（LBS)'], row['LENGTH(INCHES)'], row['WIDTH(INCHES)'], row['HEIGHT(INCHES)']), axis=1))

df.head(5)

,SKU,LENGTH(INCHES),WIDTH(INCHES),HEIGHT(INCHES),GW（LBS),One Rate Category,One Rate Cost
0,008011RNR,23.031508,23.031508,38.031517,25.08,Not Eligible,0.0
1,008013RNR,23.031508,23.031508,38.031517,25.08,Not Eligible,0.0
2,008014RNR,23.031508,23.031508,38.031517,25.08,Not Eligible,0.0
3,008026GNR-EU,23.031508,23.031508,38.031517,25.08,Not Eligible,0.0
4,008026GRS,23.031508,23.031508,38.031517,25.08,Not Eligible,0.0


### OVERWRITING CODE

In [5]:
# Write back to the same Excel file
# with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    # df.to_excel(writer, index=False, sheet_name='Sheet2')  # Adjust 'Sheet1' as necessary

In [9]:


# Load the Excel file and select the relevant sheet
file_path_excluded = 'TRNETRATES_Excluded.xlsx'  # Replace with your file path
rate_table_filtered = pd.read_excel(file_path_excluded, engine='openpyxl', sheet_name='Table 1', skiprows=164, nrows=128)

# Function to clean and convert data to numeric
def clean_numeric(data):
    return pd.to_numeric(data.replace('[^0-9.]', '', regex=True), errors='coerce')

# Extract numerical values from relevant columns
low_zone_rates_filtered = rate_table_filtered[[2, 3, 4]].apply(clean_numeric)
medium_zone_rates_filtered = rate_table_filtered[[5, 6]].apply(clean_numeric)
high_zone_rates_filtered = rate_table_filtered[[7, 8]].apply(clean_numeric)

# Extract weights
weights_filtered = clean_numeric(rate_table_filtered['lbs.'])

# Combine weights and rates into a cleaned DataFrame
cleaned_data_filtered_final = pd.DataFrame({
    'Weight (lbs)': weights_filtered,
    'Low Zone Rate': low_zone_rates_filtered.mean(axis=1, skipna=True),
    'Medium Zone Rate': medium_zone_rates_filtered.mean(axis=1, skipna=True),
    'High Zone Rate': high_zone_rates_filtered.mean(axis=1, skipna=True)
})

# Display the cleaned and filtered data
print(cleaned_data_filtered_final)


     Weight (lbs)  Low Zone Rate  Medium Zone Rate  High Zone Rate
0               1      10.700000            10.700          10.700
1               2      10.700000            10.700          10.700
2               3      10.700000            10.700          10.700
3               4      10.700000            10.700          10.700
4               5      10.700000            10.700          10.700
..            ...            ...               ...             ...
123           124      27.223333            31.460          38.920
124           125      27.223333            31.460          38.920
125           126      27.673333            31.560          39.645
126           127      28.020000            32.405          39.975
127           128      28.103333            32.510          40.295

[128 rows x 4 columns]


In [20]:
import numpy as np
# Define a function to fetch zone rates based on the rounded weight
def get_zone_rates(weight):
    # Check if the weight exceeds 71 lbs for special handling
    if weight > 71:
        # Fetch the rates for the 71-pound package
        row_71 = cleaned_data_filtered_final[cleaned_data_filtered_final['Weight (lbs)'] == 71]
        if not row_71.empty:
            low_rate_71 = row_71['Low Zone Rate'].values[0]
            medium_rate_71 = row_71['Medium Zone Rate'].values[0]
            high_rate_71 = row_71['High Zone Rate'].values[0]
            
            # Calculate the per-pound rate and then the cost for the actual weight
            low_zone_cost = (low_rate_71 / 71) * weight
            medium_zone_cost = (medium_rate_71 / 71) * weight
            high_zone_cost = (high_rate_71 / 71) * weight
            return low_zone_cost, medium_zone_cost, high_zone_cost
    
    # Regular handling for weights up to 71 lbs
    row = cleaned_data_filtered_final[cleaned_data_filtered_final['Weight (lbs)'] == weight]
    if not row.empty:
        return row['Low Zone Rate'].values[0], row['Medium Zone Rate'].values[0], row['High Zone Rate'].values[0]
    else:
        return None, None, None

# Define a function to round and ceil the weights
def round_and_ceil(weight):
    return np.ceil(weight)

# Apply the rounding and ceiling to ' GW（LBS)' and then match the rates
df['Rounded GW（LBS)'] = df[' GW（LBS)'].apply(round_and_ceil)

# Reapply the zone rates with the updated rounded and ceiled weights
df['Low Zone Cost'], df['Medium Zone Cost'], df['High Zone Cost'] = zip(*df['Rounded GW（LBS)'].apply(get_zone_rates))

# Remove the 'Rounded GW（LBS)' column before saving
df.drop(columns=['Rounded GW（LBS)'], inplace=True)


### OVERWRITE 2

In [25]:
# Write back to the same Excel file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet2')